In [1]:
# Inputs: 
# hdf5 file - HDF5 file with the specific SACLA structure
hdf5FileName = '/Users/ebner/Desktop/206178.h5'

In [2]:
import logging
logger = logging.getLogger('daqreader')

In [3]:
# variables to be read out by 'syncdaq_get' script
variables = {
    'PD': 'xfel_bl_3_st_3_pd_2_fitting_peak/voltage',
    'PD9' : 'xfel_bl_3_st_3_pd_9_fitting_peak/voltage',
    'I0' : 'xfel_bl_3_st_3_pd_4_fitting_peak/voltage',
    'M27' : 'xfel_bl_3_st3_motor_27/position',
    'M28' : 'xfel_bl_3_st3_motor_28/position',
    'LaserOn' : 'xfel_bl_lh1_shutter_1_open_valid/status',
    'LaserOff' : 'xfel_bl_lh1_shutter_1_close_valid/status',
    'Delays' : 'xfel_bl_3_st3_motor_25/position',
    'Mono' : 'xfel_bl_3_tc_mono_1_theta/position',
    'APD' : 'xfel_bl_3_st_3_pd_14_fitting_peak/voltage',
    'LasI' : 'xfel_bl3_st_3_pd_4_peak/voltage', # Extra info laser I
    'Xshut' : 'xfel_bl_3_shutter_1_open_valid/status', # X-ray on
    'Xstat' : 'xfel_mon_bpm_bl3_0_3_beamstatus/summary', # X-ray status
    'X3' : 'xfel_bl_3_st2_bm_1_pd_peak/voltage', # X-ray i 3
    'X41' : 'xfel_bl_3_st_3_pd_3_fitting_peak/voltage', # X-ray i 4
    'X42' : 'xfel_bl_3_st_3_pd_4_fitting_peak/voltage', # X-ray i 4
    'Johann' : 'xfel_bl_3_st3_motor_42/position', # Johann theta
    'APD_trans' : 'xfel_bl_3_st3_motor_17/position' # Johann det
}

In [4]:
variables

{'APD': 'xfel_bl_3_st_3_pd_14_fitting_peak/voltage',
 'APD_trans': 'xfel_bl_3_st3_motor_17/position',
 'Delays': 'xfel_bl_3_st3_motor_25/position',
 'I0': 'xfel_bl_3_st_3_pd_4_fitting_peak/voltage',
 'Johann': 'xfel_bl_3_st3_motor_42/position',
 'LasI': 'xfel_bl3_st_3_pd_4_peak/voltage',
 'LaserOff': 'xfel_bl_lh1_shutter_1_close_valid/status',
 'LaserOn': 'xfel_bl_lh1_shutter_1_open_valid/status',
 'M27': 'xfel_bl_3_st3_motor_27/position',
 'M28': 'xfel_bl_3_st3_motor_28/position',
 'Mono': 'xfel_bl_3_tc_mono_1_theta/position',
 'PD': 'xfel_bl_3_st_3_pd_2_fitting_peak/voltage',
 'PD9': 'xfel_bl_3_st_3_pd_9_fitting_peak/voltage',
 'X3': 'xfel_bl_3_st2_bm_1_pd_peak/voltage',
 'X41': 'xfel_bl_3_st_3_pd_3_fitting_peak/voltage',
 'X42': 'xfel_bl_3_st_3_pd_4_fitting_peak/voltage',
 'Xshut': 'xfel_bl_3_shutter_1_open_valid/status',
 'Xstat': 'xfel_mon_bpm_bl3_0_3_beamstatus/summary'}

In [5]:
# HDF5 metadata reading

import h5py
import re
f = h5py.File(hdf5FileName,'r')
keys = f.keys()
runs = []

# The following routine assumes following hdf5 structure
# run_<runNumber> - one for each run
# run_<runNumber>/event_info/tag_number_list - array holding all tags for specific runs
# run_<runNumber>/event_info/time_stamp - array holding all timestamps for run (inclusive start time and end time)

for k in keys:
    # find all runs inside the file
    rmatch = re.match('run_([0-9]+)', k)
    if rmatch :
        # extract run number
        rnumber = rmatch.group(1)
        print 'Matching entry: '+ rnumber
        
        # find out all tags for run
        tags = f.get(k).get('event_info').get('tag_number_list')
        
        # find out start and end timestamp of run
        timestamps = f.get(k).get('event_info').get('time_stamp')
        startTime = timestamps.value[0]
        endTime = timestamps.value[-1]
        
        structure = {'number':rnumber, 'tags': tags.value, 'startTime':startTime , 'endTime': endTime}
        
        # Code part to return an object instread of a map
        #class Struct(object):
        #    def __init__(self, **entries):
        #        self.__dict__.update(entries)        
        #runs.append(Struct(**structure))
        
        runs.append(structure)
    else:
        print('Skipping entry: ' + k)
        
print runs

Skipping entry: file_info
Matching entry: 206178
[{'endTime': 1402503460.8713069, 'number': u'206178', 'startTime': 1402503440.9101069, 'tags': array([219817020, 219817022, 219817024, 219817026, 219817028, 219817030,
       219817032, 219817034, 219817036, 219817038, 219817040, 219817042,
       219817044, 219817046, 219817048, 219817050, 219817052, 219817054,
       219817056, 219817058, 219817060, 219817062, 219817064, 219817066,
       219817068, 219817070, 219817072, 219817074, 219817076, 219817078,
       219817080, 219817082, 219817084, 219817086, 219817088, 219817090,
       219817092, 219817094, 219817096, 219817098, 219817100, 219817102,
       219817104, 219817106, 219817108, 219817110, 219817112, 219817114,
       219817116, 219817118, 219817120, 219817122, 219817124, 219817126,
       219817128, 219817130, 219817132, 219817134, 219817136, 219817138,
       219817140, 219817142, 219817144, 219817146, 219817148, 219817150,
       219817152, 219817154, 219817156, 219817158, 21

In [6]:
# determine run start and end time
import datetime
import pytz

startTime = datetime.datetime.fromtimestamp(runs[0]['startTime'], tz=pytz.timezone('Asia/Tokyo'))
endTime = datetime.datetime.fromtimestamp(runs[0]['endTime'], tz=pytz.timezone('Asia/Tokyo'))
print startTime, endTime

# to ensure that we are within the tag region by increasing the end/start timestamp by +/- 2 seconds
startTime = startTime + datetime.timedelta(seconds=-2)
endTime = endTime + datetime.timedelta(seconds=2)
print startTime, endTime

2014-06-12 01:17:20.910107+09:00 2014-06-12 01:17:40.871307+09:00
2014-06-12 01:17:18.910107+09:00 2014-06-12 01:17:42.871307+09:00


This is the syncdaq_get command to execute (for Mono):
syncdaq_get -b"2014-06-12 01:17:18.910107+09:00" -e"2014-06-12 01:17:42.871307+09:00" -f 219817020 -a 219818218  xfel_bl_3_st_3_pd_4_fitting_peak/voltage

In [7]:
# read additional information via syncdaq_get command

import subprocess
def syncdaq_get(startTime, endTime, startTag, endTag, key):
    
    #command = ['syncdaq_get', '-b', '2014-06-12 01:17:18.910107+09:00', '-e', '2014-06-12 01:17:42.871307+09:00', '-f', '219817020', '-a', '219818218', 'xfel_bl_3_st_3_pd_4_fitting_peak/voltage']
    command = ['syncdaq_get', '-b', startTime, '-e', endTime, '-f', startTag, '-a', endTag, key]
    
    proc = subprocess.Popen(command, stdout=subprocess.PIPE)
    content = proc.stdout.readlines()
    data = []
    for l in content:
        l = l.replace('\n', '')
        
        # strip characters ...
        l = l.replace('not-converged', '0')
        l = l.replace('V', '')
        l = l.replace('saturated', '0')
        l = l.replace('pulse', '')

        if l:
            (tag, value) = l.split(',')
            #print("%s %s" % (tag.strip(), value.strip()))
            data.append(value.strip())
    return data

for run in runs:
    for variable in variables:
        # Call syncdaq_get command (ideally we can retrieve data for all variables at once ...)
        try:
            print variable
            #print syncdaq_get('2014-06-12 01:17:18.910107+09:00', '2014-06-12 01:17:42.871307+09:00', '219817020', '219818218', variables[variable])
        except:
            print 'Skipping: ',variable
        

Xshut
M28
X41
PD9
LaserOff
I0
Delays
M27
LaserOn
APD_trans
Johann
X3
PD
Mono
LasI
Xstat
APD
X42


In [75]:
# retrieve stuff via syncdaq_get
# Todo: to be done
# for ii=1:length(strings)
#     try
#         File=[vars{ii} '.txt'];
    #         %command=['syncdaq_get -b"2014/06/10 04:58:27" -e"2014/06/12 23:58:32" ' strings{ii} ' | awk ''/' num2str(min(Tags)) '/{f=1} /' num2str(max(Tags)) '/{f=0;print} f'' > ' [num2str(File)]];
#         command=['syncdaq_get -b"2014/06/10 04:58:27" -e"2014/06/12 23:58:32" ' strings{ii} ' | awk ''/' num2str(min(Tags)) '/{f=1} /' num2str(max(Tags)) '/{f=0;print} f'' '];
#         [~, str] = system(command);
#        
#         str = strrep(str, 'not-converged', '0');        %# Replace wordA with wordB
#         str = strrep(str, 'V', '');
#         str = strrep(str,'saturated', '0');
#         str = strrep(str, 'pulse', '');
#         
#         %fid = fopen([File], 'w');
#         %fwrite(fid, str, '*char');              %# write characters (bytes)
#         %fclose(fid);
#         
#         %[AllTags,Stuff,~]= textread([File],'%f%f%s','delimiter',',','headerlines',1);
#         TheData = str2num(str);
#         AllTags = TheData(:,1);
#         Stuff = TheData(:,2);
#         %[AllTags,Stuff] = num2str(str);
#         
#         [val,loc,loc2]=intersect(Tags,AllTags);
#         %eval([num2str(vars{ii}) '=Stuff(loc2);'])
#         disp([num2str(vars{ii}) ' Reading succeded'])
#         DAQ.(vars{ii}) = Stuff(loc2);
#     catch
#         disp([num2str(vars{ii}) ' Reading failed'])
#     end
# end

Example output syncdaq_get command:
```
210554288, 7.22600e-02V
210554290, 1.04673e-01V
210554292, 8.31985e-02V
210554294, 1.03264e-01V
...
210554720, 7.85046e-02V
210554722, 3.33904e-02V
210554724, not-converged
210554726, 2.19994e-01V
```

provide data to leo hdf like this: {'delay':[], 'mono:[]}

In [84]:
# calculate energy based on mono position
import math

"""
Calculate beam energy based on mono position
thetaPosition - position of the mono as retrieved from the syncdaq_read command with key 'Mono'
"""
def calculateEnergy(thetaPosition):
    # Todo: Most probably these variables need to be read out from the control system ...
    thetaCoefficent = 25000 # in [pulse/mm]
    lSinbar = 275.0 # in [mm]
    thetaOffset = -15.0053431 # in [degree]
    dd = 6.270832
    
    theta = math.asin( (thetaPosition/thetaCoefficent)/lSinbar + math.sin(thetaOffset*math.pi/180.0 ) ) * 180.0/math.pi - thetaOffset;
    energy = 12.3984/((dd)*math.sin(theta*math.pi/180.0));

    return energy

Read data from hdf5 file
print f['run_206178']['daq_info']['Mono'] [:]